In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
# the symbol of human gene was designated as the name of the single-copy orthologous genes
anno = pd.read_csv('./Orthologs_to_Symbol.txt', header=None, sep=',', names=['OG', 'Symbol'])

## Positively Selection Analysis

### Ia io 
the I. io was chosen as foreground branch, with N. aviator was excluded from background branches

In [ ]:
IaIo = pd.read_csv('codeml.IaIo.results', header=0, sep='\t')
# load the w ratios estimated by using two-ratio model and one-ration model
pressure = pd.read_csv('IaIo.omega', header=0, sep='\t') 

IaIo = pd.merge(anno, IaIo, on='OG')

# the positively selected genes were identified by using Model A v.s. Model null
IaIo_psg = IaIo[IaIo.Model == 'Ma:Man']

# the rapidly evoloved gens were identified by using two-ratio model v.s. one-ratio model
IaIo_reg = IaIo[IaIo.Model == 'two:one']
IaIo_reg = pd.merge(IaIo_reg, pressure, on='OG')

In [ ]:
# significance
IaIo_psg_sig = IaIo_psg[IaIo_psg.pvalue < 0.05]
IaIo_reg_sig = IaIo_reg[(IaIo_reg.pvalue < 0.05)&(IaIo_reg.fore > IaIo_reg.back)]

In [ ]:
# save the results of psg and reg
IaIo_reg_sig.to_csv('IaIo.sig.reg.csv', header=True, index=False)
IaIo_psg_sig.to_csv('IaIo.sig.psg.csv', header=True, index=False)

### Nyatalus aviator 
the N. aviator was chosen as foreground branch, with I. io was excluded from background branches

In [11]:
NycAvi = pd.read_csv('codeml.NycAvi.results', header=0, sep='\t')
# load the w ratios estimated by using two-ratio model and one-ratio model
pressure = pd.read_csv('NycAvi.omega', header=0, sep='\t')

NycAvi = pd.merge(anno, NycAvi, on='OG')

NycAvi_reg = NycAvi[NycAvi.Model == 'two:one']
NycAvi_reg = pd.merge(NycAvi_reg, pressure, on='OG')
NycAvi_psg = NycAvi[NycAvi.Model == 'Ma:Man']

In [13]:
# significance
NycAvi_psg_sig = NycAvi_psg[NycAvi_psg.pvalue < 0.05]
NycAvi_reg_sig = NycAvi_reg[(NycAvi_reg.pvalue < 0.05)&(NycAvi_reg.fore > NycAvi_reg.back)]

In [ ]:
NycAvi_reg_sig.to_csv('IaIo.sig.reg.csv', header=True, index=False)
NycAvi_psg_sig.to_csv('NycAvi.sig.psg.csv', header=True, index=False)

## Estimated the selective pressures of genes related to immunity

Based on the key worlds to obtain gene sets related to 'adaptative immunity', 'innate immunity', and 'antiviral defense' 

In [5]:
# 'antiviral defensence'
virus = pd.read_excel('/Users/gengyang/Downloads/kw.virus.xlsx', header=0, sheet_name = 'Annotation')
virus = pd.merge(anno, virus, left_on='Symbol', right_on='hits')
virus = virus[virus['Membership: virus']==1]

In [7]:
# 'innate immunity'
innate = pd.read_excel('/Users/gengyang/Downloads/kw.innate_immune.xlsx', header=0, sheet_name = 'Annotation')
innate = innate[innate['Membership: innate immune'] == 1]
innate = pd.merge(anno, innate, left_on='Symbol', right_on='hits')

In [8]:
# 'adaptative immunity'
adaptive = pd.read_excel('/Users/gengyang/Downloads/kw.adaptative_immune.xlsx', header=0, sheet_name = 'Annotation')
adaptive = adaptive[adaptive['Membership: adaptive immune'] == 1]
adaptive = pd.merge(anno, adaptive, left_on='Symbol', right_on='hits')

load the pressures data of *I. io* and *N. aviator*

In [8]:
IaIo_omega = pd.read_csv('IaIo.omega', header=0, sep='\t')
NycAvi_omega = pd.read_csv('NycAvi.omega', header=0, sep='\t')
IaIo_omega = IaIo_omega[~((IaIo_omega.back == '-') | (IaIo_omega.fore == '-'))]
NycAvi_omega = NycAvi_omega[~((NycAvi_omega.back == '-') | (NycAvi_omega.fore == '-'))]
IaIo_omega[['back', 'fore']] = IaIo_omega[['back', 'fore']].astype(float)
NycAvi_omega[['back', 'fore']] = NycAvi_omega[['back', 'fore']].astype(float)

In [9]:
# pressures of genes linked to 'antiviral defensence'
IaIo_omega_virus = IaIo_omega[IaIo_omega.OG.isin(antiviral.OG)]
NycAvi_omega_virus = NycAvi_omega[NycAvi_omega.OG.isin(antiviral.OG)]
# pressures of genes linked to 'innate immunity'
IaIo_omega_innate = IaIo_omega[IaIo_omega.OG.isin(innate.OG)]
NycAvi_omega_innate = NycAvi_omega[NycAvi_omega.OG.isin(innate.OG)]
# pressures of genes linked to 'adaptative immunity'
IaIo_omega_adaptive = IaIo_omega[IaIo_omega.OG.isin(adaptive.OG)]
NycAvi_omega_adaptive = NycAvi_omega[NycAvi_omega.OG.isin(adaptive.OG)]

In [12]:
# the significance of changes of 'adaptative immunity'
stats.ttest_ind(IaIo_omega_adaptive.fore, IaIo_omega_adaptive.back, alternative='greater', equal_var=False)
stats.ttest_ind(NycAvi_omega_adaptive.fore, NycAvi_omega_adaptive.back, alternative='greater', equal_var=False)

TtestResult(statistic=2.060661748192677, pvalue=0.020247801486661676, df=223.00094684883672)

In [13]:
# the significance of changes of 'innate immunity'
stats.ttest_ind(NycAvi_omega_adaptive.fore, NycAvi_omega_adaptive.back, alternative='greater', equal_var=False)
stats.ttest_ind(NycAvi_omega_innate.fore, NycAvi_omega_innate.back, alternative='greater', equal_var=False)

TtestResult(statistic=1.7461683170778253, pvalue=0.041079366938232005, df=223.00125911463707)

In [165]:
# the significance of changes of 'antiviral defense'
stats.ttest_ind(IaIo_omega_virus.fore, IaIo_omega_virus.back, alternative='greater', equal_var=True)
stats.ttest_ind(NycAvi_omega_virus.fore, NycAvi_omega_virus.back, alternative='greater', equal_var=True)

TtestResult(statistic=1.4910312026075163, pvalue=0.06815145889101036, df=902.0)

In [17]:
# save the results
IaIo_omega_innate.to_csv('IaIo_omega_innate.tsv', header=True, index=False, sep='\t')
NycAvi_omega_innate.to_csv('NycAvi_omega_innate.tsv', header=True, index=False, sep='\t')

IaIo_omega_adaptive.to_csv('IaIo_omega_adaptive.tsv', header=True, index=False, sep='\t')
NycAvi_omega_adaptive.to_csv('NycAvi_omega_adaptive.tsv', header=True, index=False, sep='\t')

IaIo_omega_virus.to_csv('IaIo_omega_virus.tsv', header=True, index=False, sep='\t')
NycAvi_omega_virus.to_csv('NycAvi_omega_virus.tsv', header=True, index=False, sep='\t')

## RERconvergent analysis

In [4]:
RER = pd.read_csv('RERconverge.results', header=0, sep='\t')
RER = pd.merge(anno, RER, left_on='OG', right_on='gene')
RER.drop(columns='gene', inplace=True)

In [6]:
RER.to_csv('avivorous.RER.csv', header=True, index=False)

## Identified convergent amino acid substitutations by using Zou and Zhang's method

In [3]:
# the convergent amino acid substitutations identified under f-site and f-gene model, respectively
# f-site model
site = pd.read_csv('conv_site.tsv', header=0, sep='\t')
site = pd.merge(anno, site, on='OG')
# f-gene model
gene = pd.read_csv('conv_gene.tsv', header=0, sep='\t')
gene = pd.merge(anno, gene, on='OG')

In [4]:
# poisson test 
site['p_sig'] = site.apply(lambda x: stats.poisson.pmf(x['parallel_obs'], x['parallel_exp']), axis=1)
site['c_sig'] = site.apply(lambda x: stats.poisson.pmf(x['converge_obs'], x['converge_exp']), axis=1)

gene['p_sig'] = site.apply(lambda x: stats.poisson.pmf(x['parallel_obs'], x['parallel_exp']), axis=1)
gene['c_sig'] = site.apply(lambda x: stats.poisson.pmf(x['converge_obs'], x['converge_exp']), axis=1)

In [5]:
# Finally, we have chosen the result of the f-site model as the final result.
site_bird['p_fdr'] = stats.false_discovery_control(site_bird.p_sig)
site_bird['c_fdr'] = stats.false_discovery_control(site_bird.c_sig)

bird_parallel = site[(site.group == 'NycAvi,IaIo') & (site.p_sig < 0.05) & (site.parallel_obs > 0)]
bird_converge = site[(site.group == 'NycAvi,IaIo') & (site.c_sig < 0.05) & (site.converge_obs > 0)]

In [6]:
# save final result
site_bird[site_bird.p_fdr < 0.05].to_csv('avivorous.conv.site.csv', header=True, index=False)

In [ ]:
# the intersection between results of RERconvergence and conv_cale
RER[RER.Symbol.isin(['APOA1', 'CPT1B', 'CPT2', 'CYP7A1', 'ANGPTL4', 'CEPT1'])]

##  Gene Family Evolution

In [13]:
# load data of significant evoloved gene families in N. aviator
NycAvi_sig_exp = pd.read_csv('NycAvi_sig_exp.tsv', header=0, sep=',')
NycAvi_sig_con = pd.read_csv('NycAvi_sig_con.tsv', header=0, sep=',')

# load data of significant evoloved gene families in I. io
IaIo_sig_exp = pd.read_csv('IaIo_sig_exp.tsv', header=0, sep=',')
IaIo_sig_con = pd.read_csv('IaIo_sig_con.tsv', header=0, sep=',')

##  Gene Loss Identification

In [212]:
# load the results of TOGA
IaIo_loss =   pd.read_csv('Iaio.loss_summ_data.tsv',   header=None, names=['type', 'gene', 'IaIo'],   sep='\t')
NycAvi_loss = pd.read_csv('NycAvi.loss_summ_data.tsv', header=None, names=['type', 'gene', 'NycAvi'], sep='\t')
RouAeg_loss = pd.read_csv('RouAeg.loss_summ_data.tsv', header=None, names=['type', 'gene', 'RouAeg'], sep='\t')
PteAle_loss = pd.read_csv('PteAle.loss_summ_data.tsv', header=None, names=['type', 'gene', 'PteAle'], sep='\t')
HipArm_loss = pd.read_csv('HipArm.loss_summ_data.tsv', header=None, names=['type', 'gene', 'HipArm'], sep='\t')
RhiFer_loss = pd.read_csv('RhiFer.loss_summ_data.tsv', header=None, names=['type', 'gene', 'RhiFer'], sep='\t')
DesRot_loss = pd.read_csv('DesRot.loss_summ_data.tsv', header=None, names=['type', 'gene', 'DesRot'], sep='\t')
PhyDis_loss = pd.read_csv('PhyDis.loss_summ_data.tsv', header=None, names=['type', 'gene', 'PhyDis'], sep='\t')
ArtJam_loss = pd.read_csv('ArtJam.loss_summ_data.tsv', header=None, names=['type', 'gene', 'ArtJam'], sep='\t')
MolMol_loss = pd.read_csv('MolMol.loss_summ_data.tsv', header=None, names=['type', 'gene', 'MolMol'], sep='\t')
MinNat_loss = pd.read_csv('MinNat.loss_summ_data.tsv', header=None, names=['type', 'gene', 'MinNat'], sep='\t')
MyoMyo_loss = pd.read_csv('MyoMyo.loss_summ_data.tsv', header=None, names=['type', 'gene', 'MyoMyo'], sep='\t')
MyoDav_loss = pd.read_csv('MyoDav.loss_summ_data.tsv', header=None, names=['type', 'gene', 'MyoDav'], sep='\t')
PipKuh_loss = pd.read_csv('PipKuh.loss_summ_data.tsv', header=None, names=['type', 'gene', 'PipKuh'], sep='\t')
EptFus_loss = pd.read_csv('EptFus.loss_summ_data.tsv', header=None, names=['type', 'gene', 'EptFus'], sep='\t')

In [215]:
# filter the gene loss events that are only present in I. io and N. aviator
bat_loss = IaIo_loss
for i in [NycAvi_loss, RouAeg_loss, PteAle_loss, HipArm_loss, RhiFer_loss, DesRot_loss, PhyDis_loss, ArtJam_loss, MolMol_loss, MinNat_loss, MyoMyo_loss, MyoDav_loss, PipKuh_loss, EptFus_loss]:
    bat_loss = pd.merge(bat_loss, i, on=['type', 'gene'])
bat_loss = bat_loss[bat_loss.type == 'GENE']
bat_loss[(bat_loss.apply(lambda x: len(x[x=='L']), axis = 1)==2) & (bat_loss.IaIo == 'L') & (bat_loss.NycAvi == 'L')]

In [197]:
IaIo_loss = IaIo_loss[(IaIo_loss.type == 'GENE') & (IaIo_loss.stats == 'L')]
NycAvi_loss = NycAvi_loss[(NycAvi_loss.type == 'GENE') & (NycAvi_loss.stats == 'L')]